In [1]:
%matplotlib inline

import cv2
import matplotlib.pyplot as plt
from cv2.ximgproc import guidedFilter
import numpy as np
from skimage.filters import sobel_v,sobel_h, sobel
import skimage 
import warnings
import skimage.io
import skimage.morphology as morph
from ipywidgets import interact 

warnings.filterwarnings('ignore')

plt.rcParams["figure.figsize"] = (12, 5)
plt.rcParams['ytick.left'] = False
plt.rcParams['ytick.labelleft'] = False
plt.rcParams['xtick.bottom'] = False
plt.rcParams['xtick.labelbottom'] = False
plt.rcParams.update({'font.size': 18})


def read(num, mode = 'color'):
    path = 'img_duvan/' + str(num)
    img = cv2.cvtColor(cv2.imread(path+'.jpg'),cv2.COLOR_BGR2RGB)
    img = img[:,260:1120,]
    return img

def plot(img):
    plt.figure()
    plt.imshow(img, cmap='gray')
    plt.show()
    return None 

In [16]:
def fun_img(id, limit):
    img = read(id)

    hsv  = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    yuv  = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)

    img_s  =  hsv[:,:,1]
    img_v  =  hsv[:,:,2]
    img_y  =  yuv[:,:,0]

    _, borde = cv2.threshold(img_v,70,255, cv2.THRESH_BINARY)
    
    
    #TOTAS LAS CELULAS
    
    umbral =cv2.adaptiveThreshold(img_y,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 21, 5) 
    umbral = umbral * borde

    kernel = morph.disk(1)
    apertura = cv2.morphologyEx(umbral, cv2.MORPH_OPEN, kernel, iterations = 1 )
    
    kernel = morph.disk(3)
    cierre   = cv2.morphologyEx(apertura, cv2.MORPH_CLOSE, kernel, iterations = 3)
    
    kernel = morph.disk(2)
    apertura = cv2.morphologyEx(cierre, cv2.MORPH_OPEN, kernel, iterations = 4 )    

    llenado = morph.remove_small_holes(apertura,area_threshold=4000) #min_size es el area en pixeles de los huecos a rellenar
    llenado = llenado.astype(np.uint8)

    contours, hierarchy = cv2.findContours(llenado.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    img_cont = cv2.drawContours(img.copy(), contours, -1, (255,0,0), 3)
    
    area_1 = []
    for i in range(len(contours)):
        a = cv2.contourArea(contours[i])
        area_1.append(a)
        tot_1 = sum(area_1)
    print('Area total de células= ',tot_1)
   
    
    #SÓLO LAS CÉLULAS NO VIABLES
    
    mask_clahe = img_s * llenado
    clahe1 = cv2.createCLAHE(clipLimit=20.0, tileGridSize=(10,10))
    cl1 = clahe1.apply(mask_clahe)
    
    _, umbral_2 = cv2.threshold(cl1,limit,255, cv2.THRESH_BINARY)
    umbral_2 = umbral_2*borde

    kernel = morph.disk(1)
    apertura_2 = cv2.morphologyEx(umbral_2, cv2.MORPH_OPEN, kernel, iterations = 2 )

    kernel = morph.disk(3)
    cierre_2   = cv2.morphologyEx(apertura_2, cv2.MORPH_CLOSE, kernel, iterations = 6)
    #plot(cierre_2)

    kernel = morph.disk(4)
    dilation_2 = cv2.dilate(cierre_2,kernel,iterations = 2)
    dilation_2 = dilation_2 * borde

    contours_2, hierarchy = cv2.findContours(dilation_2.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    img_cont_2 = cv2.drawContours(img.copy(), contours_2, -1, (255,0,0), 3)
    
    area_2 = []
    for i in range(len(contours_2)):
        a = cv2.contourArea(contours_2[i])
        area_2.append(a)
        tot_2 = sum(area_2)
    print('Area total de células= ',tot_2,)
    
    viabilidad = round((((tot_1-tot_2)/tot_1)*100),2)

    print('Viabilidad=', viabilidad, '%', end='')
    return img, llenado, dilation_2, img_cont, img_cont_2, viabilidad



def graf(id=10, limit=170):
    img, llenado, llenado_2, img_cont, img_cont_2, area = fun_img(id, limit)
    fig, ax = plt.subplots(1,3, figsize=(25,7))
    ax[0].imshow(img)
    ax[0].set_title('Imagen Original\n')
    ax[1].imshow(llenado,cmap='gray')
    ax[1].set_title('Máscara\n')
    ax[2].imshow(img_cont)
    ax[2].set_title('Resultado\n')
    plt.show()

    fig, ax = plt.subplots(1,3, figsize=(25,7))
    ax[0].imshow(img)
    ax[1].imshow(llenado_2,cmap='gray')
    ax[2].imshow(img_cont_2)
    plt.show()
       
interact(graf, 
            id     = (1, 20, 1),
            umbral = (100, 250, 10),
        )
    
None


interactive(children=(IntSlider(value=10, description='id', max=20, min=1), IntSlider(value=170, description='…